# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Необходимо проанализировать, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.  
Входные данные от банка — статистика о платёжеспособности клиентов.  
Результаты исследования будут учтены при построении модели кредитного скоринга.


## Общая информация о датасете




Импортируем необходимые библиотеки.  
Датафрейм сохраним в переменной `data`.

In [ ]:
import pandas as pd

In [ ]:
try:
    data = pd.read_csv('/content/sample_data/credit_score.csv')
except:
    print('Нет доступа к файлу')

Выведем случайные пять строк датасета.

In [ ]:
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
7956,0,-7699.940954,65,среднее,1,женат / замужем,0,F,госслужащий,0,163471.594051,покупка коммерческой недвижимости
4184,0,376617.584253,57,среднее,1,женат / замужем,0,F,пенсионер,0,165644.100462,дополнительное образование
17061,1,-476.384839,24,среднее,1,женат / замужем,0,M,сотрудник,0,184535.209613,покупка жилой недвижимости
19718,0,384997.639578,67,среднее,1,гражданский брак,1,M,пенсионер,0,40217.934048,операции со своей недвижимостью
13364,3,-160.349105,35,среднее,1,женат / замужем,0,M,сотрудник,0,190624.722427,строительство жилой недвижимости


In [ ]:
num_rows = data.shape[0]

**Описание данных**

- `children` — количество детей в семье;
- `days_employed` — общий трудовой стаж в днях;
- `dob_years` — возраст клиента в годах;
- `education` — уровень образования клиента;
- `education_id` — идентификатор уровня образования;
- `family_status` — семейное положение;
- `family_status_id` — идентификатор семейного положения;
- `gender` — пол клиента;
- `income_type` — тип занятости;
- `debt` — имел ли задолженность по возврату кредитов;
- `total_income` — ежемесячный доход;
- `purpose` — цель получения кредита.

Выведем основную информацию о датафрейме.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Выводы по разделу

В таблице содержится информация о 21525 клиентах, подавших заявку на получение кредита. Для каждого клиента предоставлена информация о количестве детей, продолжительности трудового стажа, семейном положении, уровне образования, доходе и допущенных задолженностях.
После первичного ознакомления с данными выявлено, что в столбцах `days_employed` и `total_income` встречаются пропуски. Количество пропущенных значений в этих столбцах одинаковое, что может указывать на связь между этими пропусками. В столбцах `children` и `dob_years` присутствуют аномальные значения (-1, 20 для количества детей и 0 для возраста). В столбце `days_employed` есть отрицательные значения.

Необходимо выполнить следующие шаги:

- проверить датасет на наличие дубликатов;
- устранить аномальные значения столбцов `children` и `dob_years`;
- проанализировать столбец `days_employed` и определить причину наличия отрицательных значений;
- устранить пропуски данных в столбцах `days_employed` и `total_income`;
- стандартизировать текстовые данные, при необходимости.

В рамках проекта предстоит ответить на следующие шесть вопросов:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?
5. Приведите возможные причины появления пропусков в исходных данных.
6. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

## Предобработка данных

### Проверка уникальных значений в столбцах с данными типа object

Выведем уникальные значения для столбцов с данными о семейном положении, количестве детей, образовании, поле, типе дохода и наличии задолженности.

In [ ]:
# Создадим список интересующих нас столбцов
lst = ['children', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt']

# Выведем уникальные значения для каждого
for i in lst:
    unique_values = ', '.join(map(str, data[i].unique()))
    print(f'Уникальные значения столбца {i}: {unique_values}')

Уникальные значения столбца children: 1, 0, 3, 2, -1, 4, 20, 5
Уникальные значения столбца education: высшее, среднее, Среднее, СРЕДНЕЕ, ВЫСШЕЕ, неоконченное высшее, начальное, Высшее, НЕОКОНЧЕННОЕ ВЫСШЕЕ, Неоконченное высшее, НАЧАЛЬНОЕ, Начальное, Ученая степень, УЧЕНАЯ СТЕПЕНЬ, ученая степень
Уникальные значения столбца education_id: 0, 1, 2, 3, 4
Уникальные значения столбца family_status: женат / замужем, гражданский брак, вдовец / вдова, в разводе, Не женат / не замужем
Уникальные значения столбца family_status_id: 0, 1, 2, 3, 4
Уникальные значения столбца gender: F, M, XNA
Уникальные значения столбца income_type: сотрудник, пенсионер, компаньон, госслужащий, безработный, предприниматель, студент, в декрете
Уникальные значения столбца debt: 0, 1


In [ ]:
minus_one_chld = data[data['children'] == -1].count()[0]
twenty_chld = data[data['children'] == 20].count()[0]

print(f'Количество записей "-1" в столбце "children" - {minus_one_chld} ({round(minus_one_chld/num_rows * 100, 3)}% от общего числа).')
print(f'Количество записей "20" в столбце "children" - {twenty_chld} ({round(twenty_chld/num_rows * 100, 3)}% от общего числа).')

Количество записей "-1" в столбце "children" - 47 (0.218% от общего числа).
Количество записей "20" в столбце "children" - 76 (0.353% от общего числа).


В столбце `children` встречается 47 записей со значением "-1" и 76 записей со значением "20".  
Можно предположить, что их возникновение связано с ошибкой при заполнении данных. Так как восстановить действительные данные не представляется возможным и количество аномальных значений не превышает 0.6% удалим строки с аномальными значениями.

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Выведем уникальные значения столбца `chuildren` для проверки корректности удаления.

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Стандартизируем текст в столбце `education` - приведем все значения к нижнему регистру.

In [ ]:
data['education'] = data['education'].str.lower()

<ipython-input-171-d4b849f854d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['education'] = data['education'].str.lower()


In [ ]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

В столбце `gender` встречается запись "XNA". Выведем строки с данной записью.

In [ ]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Установить пол по известным данным не представляется возможным. Так как это значение содержит лишь одна строка, можно отнести ее к любому полу или просто удалить.

In [ ]:
data = data[data['gender'] != 'XNA']

### Обработка дубликатов

Выведем на экран количество строк-дубликатов в данных и удалим их, если они присутствуют.

In [ ]:
print(f'Количество дубликатов - {data.duplicated().sum()}.')

Количество дубликатов - 71.


In [ ]:
data = data.drop_duplicates()

Убедимся, что дубликатов не осталось.

In [ ]:
data.duplicated().sum()

0

### Удаление пропусков

Выведем количество пропущенных значений.

In [ ]:
# Подсчет количества пропусков в каждом столбце
missing_values_count = data.isnull().sum()

# Сохраним исходный датафрейм в отдельную переменную
data_with_null = data.copy()

# Подсчет процентного соотношения пропусков в каждом столбце
missing_values_percentage = (data.isnull().sum() / len(data)) * 100

# Соединяем оба показателя в один DataFrame для удобства анализа
missing_values_analysis = pd.DataFrame({'Количество пропусков': missing_values_count,
                                        'Процент пропусков': missing_values_percentage})

missing_values_analysis[missing_values_analysis['Количество пропусков'] > 0]


,Количество пропусков,Процент пропусков
days_employed,2091,9.803094
total_income,2091,9.803094


Проанализируем распределение пропусков по разным категориям.

In [ ]:
# Определение строк с пропущенными значениями
rows_with_missing = data[data['days_employed'].isnull() | data['total_income'].isnull()]

# Сравнение распределения категорий с пропусками и без пропусков
categories = ['income_type', 'education', 'family_status', 'gender']
missing_distributions = {category: rows_with_missing[category].value_counts(normalize=True) for category in categories}
overall_distributions = {category: data[category].value_counts(normalize=True) for category in categories}

# Создание DataFrame для сравнения распределений
comparison_dfs = {category: pd.DataFrame({'Overall': overall_distributions[category],
                                          'Missing': missing_distributions[category]}) for category in categories}

comparison_dfs['income_type']


,Overall,Missing
безработный,0.000094,NaN
в декрете,0.000047,NaN
госслужащий,0.068026,0.068867
компаньон,0.236568,0.237685
пенсионер,0.178715,0.183166
предприниматель,0.000094,0.000478
сотрудник,0.516409,0.509804
студент,0.000047,NaN


Процентное распределение по типам дохода в целом по датасету (Overall) и в строках с пропущенными значениями (Missing) довольно похоже.  
В категории "госслужащий", "компаньон" и "сотрудник" доля пропущенных значений соответствует общему распределению этих категорий в датасете.
Немного выше доля пропущенных значений среди "пенсионеров" (18.3% пропущенных против 17.9% в общем распределении) и "предпринимателей" (0.048% против 0.0094%).  
Категории "безработный", "в декрете" и "студент" не имеют пропущенных значений, хотя их доля в общем распределении очень мала.  
Эти наблюдения могут указывать на то, что пропуски в данных не сильно зависят от типа занятости респондентов. Вероятно, пропуски обусловлены другими факторами, не связанными непосредственно с профессиональной деятельностью.  
На основе вышесказанного, заполним пропуски медианными значениями по типу дохода. Это позволит сохранить структуру данных без значительного искажения общей картины.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В столбце `days_employed` встречаются аномалии — отрицательное количество дней трудового стажа.  
Сгруппируем по типу дохода и подсчитаем количество отрицательных и положительных значений в days_employed.

In [ ]:
grouped_data = data.groupby('income_type')['days_employed'].agg(
    positive_count=lambda x: (x > 0).sum(),
    negative_count=lambda x: (x < 0).sum()
).reset_index()

grouped_data

,income_type,positive_count,negative_count
0,безработный,2,0
1,в декрете,0,1
2,госслужащий,0,1307
3,компаньон,0,4549
4,пенсионер,3429,0
5,предприниматель,0,1
6,сотрудник,0,9949
7,студент,0,1


Положительные значения трудового стажа встречаются только у пенсионеров и безработных. Вероятно, отрицательные значения появились ошибочно.  
Заменим отрицательные значения положительными.

In [ ]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed`.

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.137274
компаньон            1555.901115
пенсионер          365286.622650
предприниматель       520.848083
сотрудник            1572.328285
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Похоже, что для этих двух типов стаж был указан в часах (для пенсионеров 365286 / 24 / 365 = 41,7 лет).  
В рамках текущей задачи нам эти значения не понадобятся, поэтому просто присвоим пустым значениям `days_employed` соответствующие медианные значения.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заменены.

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбцах `total_income` и `days_employed` на целочисленный.

In [ ]:
data['total_income'] = data['total_income'].astype(int)
data['days_employed'] = data['days_employed'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21330 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21330 non-null  int64 
 1   days_employed     21330 non-null  int64 
 2   dob_years         21330 non-null  int64 
 3   education         21330 non-null  object
 4   education_id      21330 non-null  int64 
 5   family_status     21330 non-null  object
 6   family_status_id  21330 non-null  int64 
 7   gender            21330 non-null  object
 8   income_type       21330 non-null  object
 9   debt              21330 non-null  int64 
 10  total_income      21330 non-null  int64 
 11  purpose           21330 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Лемматизация целей кредта

Для упрощения дальнейшего анализа разобъем все цели кредита на группы.  
Выведем все уникальные значния столбца `purpose`.

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

На основании данных из столбца `purpose` сформируем новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
4315,0,1087,31,среднее,1,женат / замужем,0,F,сотрудник,0,162948,жилье,операции с недвижимостью
18925,0,3441,43,среднее,1,женат / замужем,0,M,сотрудник,0,79887,заняться высшим образованием,получение образования
4025,0,235,48,среднее,1,Не женат / не замужем,4,F,сотрудник,0,84455,автомобиль,операции с автомобилем
18544,1,376,29,неоконченное высшее,2,гражданский брак,1,M,сотрудник,0,222991,на проведение свадьбы,проведение свадьбы
18364,0,2987,54,среднее,1,женат / замужем,0,F,компаньон,0,43040,покупка жилья,операции с недвижимостью


### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'Низкий доход'`;
- 30001–50000 — `'Ниже среднего'`;
- 50001–200000 — `'Средний доход'`;
- 200001–1000000 — `'Выше среднего'`;
- 1000001 и выше — `'Высокий доход'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'Низкий доход'
        elif 30001 <= income <= 50000:
            return 'Ниже среднего'
        elif 50001 <= income <= 200000:
            return 'Средний доход'
        elif 200001 <= income <= 1000000:
            return 'Выше среднего'
        elif income >= 1000001:
            return 'Высокий доход'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

## Выоды по разделу

## Ответы на вопросы

### 1. Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для выявления взаимосвязи объединим данные в группы по количесвтву детей и для каждой группы рассчитаем средний показатель долга.

In [ ]:
#Вычислим общее количество клиентов, сумму клиентов с задолженностями и среднее значение долга по каждой категории.
data_pivot_children = data.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', 'mean'])

#Рассчитаем процент задолженностей и процент заеищиков по каждой категории от общего числа и выведем общую таблицу
total_count = data.shape[0]
data_pivot_children['percent'] = (data_pivot_children['sum']['debt'] / data_pivot_children['count']['debt'] * 100).round(1)
data_pivot_children['share_group'] = (data_pivot_children['count']['debt'] / total_count * 100).round(1)
data_pivot_children

,count,sum,mean,percent,share_group
,debt,debt,debt,,
children,,,,,
0,14090,1063,0.075444,7.5,66.1
1,4808,444,0.092346,9.2,22.5
2,2052,194,0.094542,9.5,9.6
3,330,27,0.081818,8.2,1.5
4,41,4,0.097561,9.8,0.2
5,9,0,0.000000,0.0,0.0


**Вывод:** Исходя из полученных данных можно сказать что заемщики без детей (составляют 66,1% от общего числа заемщиков) имеют меньше просрочек по выплате кредита в сравнении с другими группами - 7,5%. В то время как заемщики с одним и двумя детьми чаще допускают просрочки - 9,2% и 9,5% соответственно. В связи с небольшим количеством многодетных клиентов в выборке по ним сложно сделать определенные выводы.

### 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для выявления взаимосвязи сгруппируем данные по семейному положению и для каждой группы рассчитаем средний показатель долга.

In [ ]:
#Вычислим общее количество клиентов, сумму клиентов с задолженностями и среднее значение долга по каждой категории.
data_pivot_marriage = data.pivot_table(index='family_status', values='debt', aggfunc=['count', 'sum', 'mean'])

In [ ]:
#Рассчитаем процент задолженностей и процент заеищиков по каждой категории от общего числа и выведем общую таблицу
data_pivot_marriage['percent'] = (data_pivot_marriage['sum']['debt'] / data_pivot_marriage['count']['debt'] * 100).round(1)
data_pivot_marriage['share_group'] = (data_pivot_marriage['count']['debt'] / total_count * 100).round(1)
data_pivot_marriage

,count,sum,mean,percent,share_group
,debt,debt,debt,,
family_status,,,,,
Не женат / не замужем,2796,273,0.097639,9.8,13.1
в разводе,1189,84,0.070648,7.1,5.6
вдовец / вдова,951,63,0.066246,6.6,4.5
гражданский брак,4133,385,0.093153,9.3,19.4
женат / замужем,12261,927,0.075606,7.6,57.5


**Вывод:** Согласно среднему показателю долга меньше просрочек по выплатам кредита допускают овдовевшие (6,6%), развведенные (7,1%) и женатые/замужние (7,6%) заемщики. При этом, стоит отметить, что заемщики, состоящие в браке составляют более 57,5%  от общего количества, в то время как разведенные и овдовевшие - 5,6% и 4,5% соответственно. Люди, не состоящие в браке или проживающие в гражданском браке, в свою очередь, чаще нарушают сроки возврата кредита (9,8% и 9,3%).

### 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для выявления взаимосвязи сгруппируем данные по уровню дохода и для каждой группы рассчитаем средний показатель долга.

In [ ]:
#Вычислим общее количество клиентов, сумму клиентов с задолженностями и среднее значение долга по каждой категории.
data_pivot_income = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum', 'mean'])

#Рассчитаем процент задолженностей и процент заеищиков по каждой категории от общего числа и выведем общую таблицу
data_pivot_income['percent'] = (data_pivot_income['sum']['debt'] / data_pivot_income['count']['debt'] * 100).round(1)
data_pivot_income['share_group'] = (data_pivot_income['count']['debt'] / total_count * 100).round(1)
data_pivot_income

,count,sum,mean,percent,share_group
,debt,debt,debt,,
total_income_category,,,,,
Высокий доход,25,2,0.080000,8.0,0.1
Выше среднего,5013,354,0.070616,7.1,23.5
Ниже среднего,349,21,0.060172,6.0,1.6
Низкий доход,22,2,0.090909,9.1,0.1
Средний доход,15921,1353,0.084982,8.5,74.6


**Предварительный вывод:** Больую часть составляют заемщики с уровнем дохода от 50.000 до 200.000 рублей (74,6%). При этом они допускают просрочки чаще, чем заемщики других категорий. За исключением заемщиков с уровнем дохода до 30.000 (клиенты с низким доходом) рублей, просрочка выплат которых состовляет 9,1%. Однако данная группа составляет менее 1,0% от общего числа заемщиков и не является показательной. Меньше всего просрочек у группы с уровнем дрхода 30.001 - 50.000 рублей - 6,0%, но данная группа составляет лишь 1,6% от общего числа.
В связи с сильным отклонением численности группы 'С' необходимо пересмотреть её диапазон группировки доходов. Разделить группу 'C' на несколько групп (50.001 - 100.000, 100.001 - 150.000, 150.001 - 200.000) и повторить расчет.

In [ ]:
#Добавим дополнительные категории для группировки
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'До 30к'
        elif 30001 <= income <= 50000:
            return 'От 30к до 50к'
        elif 50001 <= income <= 100000:
            return 'От 50к до 100к'
        elif 100001 <= income <= 150000:
            return 'От 100к до 150к'
        elif 150001 <= income <= 200000:
            return 'От 150к до 200к'
        elif 200001 <= income <= 1000000:
            return 'От 200к до 1млн'
        elif income >= 1000001:
            return 'Свыше 1 млн'
    except:
        pass

data['total_income_category'] = data['total_income'].apply(categorize_income)

In [ ]:
#повторим расчет
data_pivot_income = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum', 'mean'])
data_pivot_income['percent'] = (data_pivot_income['sum']['debt'] / data_pivot_income['count']['debt'] * 100).round(1)
data_pivot_income['share_group'] = (data_pivot_income['count']['debt'] / total_count * 100).round(1)
data_pivot_income

,count,sum,mean,percent,share_group
,debt,debt,debt,,
total_income_category,,,,,
До 30к,22,2,0.090909,9.1,0.1
От 100к до 150к,7110,619,0.087060,8.7,33.3
От 150к до 200к,4738,403,0.085057,8.5,22.2
От 200к до 1млн,5013,354,0.070616,7.1,23.5
От 30к до 50к,349,21,0.060172,6.0,1.6
От 50к до 100к,4073,331,0.081267,8.1,19.1
Свыше 1 млн,25,2,0.080000,8.0,0.1


**Вывод:** После перегруппировки данных заметно, что заемщики групп 100.001 - 150.000 и 150.001 - 200.000 допускают просрочки чаще, чем из группы 'E'. Однако, группа 'E' по прежнему имеет высокий показатель несвоевременного возврата кредита.  
Исходя из здравого смысла логично предположить, что чаще всего должниками становятся люди с небольшим доходом, которым финансовые проблемы не позволяют вовремя осуществлять платежи. Но принимая во внимание наши результаты стоит отметить, что выборка по заемщикам, как с наименьшим доходом, так и с наибольшим крайне недостаточна для формирования однозначных выводов. Можно думать, что самая низкооплачиваемая не может корректно распоряжаться деньгами и «влезает» в кредиты, но тогда неясно почему у клиентов с высоким доходом также достаточно высокий процент задолженности.

Если же взять для анализа только остальные группы, то по полученным данным не наблюдаем зависимости "те, кто получает больше ЗП, лучше возвращает кредит». В целом это логично, так как материальный достаток - не единственная переменная, которая влияет на качество кредитной истории.

Стоит принять во внимание 2174 пропуска в данных столбца "total_income" заполненные медианным значением по типу занятости. Сводная таблица с данными о зависимости между уровнем дохода и возвратом кредита в срок возможно могла бы выглядеть иначе, если бы не пропущенные данные.

### 4. Как разные цели кредита влияют на его возврат в срок?

Для выявления взаимосвязи сгруппируем данные по цели кредита и для каждой группы рассчитаем средний показатель долга.

In [ ]:
data_pivot_purpose = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum', 'mean'])

data_pivot_purpose['percent'] = (data_pivot_purpose['sum']['debt'] / data_pivot_purpose['count']['debt'] * 100).round(1)
data_pivot_purpose['share_group'] = (data_pivot_purpose['count']['debt'] / total_count * 100).round(1)
data_pivot_purpose

,count,sum,mean,percent,share_group
,debt,debt,debt,,
purpose_category,,,,,
операции с автомобилем,4279,400,0.093480,9.3,20.1
операции с недвижимостью,10750,780,0.072558,7.3,50.4
получение образования,3988,369,0.092528,9.3,18.7
проведение свадьбы,2313,183,0.079118,7.9,10.8


**Вывод:** Наибольшее число просроченных выплат по кредитам приходится на автокредиты и кредиты на образование - 9,3%. Просрочек по кредитам на недвижимость значительно меньше - 7,3%, при их оотносительной численности в 50,4% от всех займов.

Также, при необходимости, можно рассмотреть кредиты на приобретение коммерческой и жилой недвижимости по отдельности.

### 5. Приведите возможные причины появления пропусков в исходных данных.

In [ ]:
data_with_null.groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
безработный,2,2,2,2,2,2,2,2,2,2,2
в декрете,1,1,1,1,1,1,1,1,1,1,1
госслужащий,1451,1307,1451,1451,1451,1451,1451,1451,1451,1307,1451
компаньон,5046,4549,5046,5046,5046,5046,5046,5046,5046,4549,5046
пенсионер,3812,3429,3812,3812,3812,3812,3812,3812,3812,3429,3812
предприниматель,2,1,2,2,2,2,2,2,2,1,2
сотрудник,11015,9949,11015,11015,11015,11015,11015,11015,11015,9949,11015
студент,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
data_with_null[data_with_null['total_income'].isna()].groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
госслужащий,144,0,144,144,144,144,144,144,144,0,144
компаньон,497,0,497,497,497,497,497,497,497,0,497
пенсионер,383,0,383,383,383,383,383,383,383,0,383
предприниматель,1,0,1,1,1,1,1,1,1,0,1
сотрудник,1066,0,1066,1066,1066,1066,1066,1066,1066,0,1066


Пропущенные знвчения составляют около 10% от общего числа записей. При этом пропуски в столбцах 'days_employed' и 'total_income' находятся в одних и тех же строках, а также отчасти зависят друг от друга.  <br />    При этом, если изучть количество пропуков по каждому типу занятости, то заметно, что отсутствующие данные в каждой группе также составляют 10%. <br /   > Допускаю несколько возможных причин возникновения пропусков. <br />   
       1. Так как данные отсутствуют "равномерно" по всем категориям, необходимо проверить источник их получения. Возможно, данные хранились с разным форматированием (разное название колонок или порядок их расположения) и это не было выявлено при их объединении, что привело к потере части информации.
    <br />2. В случае, если при заполнении информации в системе не стоит проверка на обязательность заполнения ячейки, данная информация могла быть пропущена кредитным менеджером. Но в связи с достаточно высоким процентом пропусков данный вариант считаю маловероятным.  </font>


### 6. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Заполнение пропусков медианными значениями подходит в случае неравномерного распределения данных и позволяет определить центральное значение данных. Если в данных наблюдаются "выбросы", то даже небольшое их количество может сильно "перетягивать" среднее значение в большую или меньшую сторону. Среднее арифметическое подходит в случае нормального распределения данных.

## Общий вывод

Удаление небольшого количества аномальных значений в столбце children было оправдано из-за их невозможности коррекции или восстановления. Это решение улучшило качество данных, минимизировав искажение общей картины.

Было также замечено, что процентное распределение по типам дохода среди пропущенных значений не показывает значительного отклонения от общего распределения в датасете. Это указывает на отсутствие явной зависимости пропусков от типа занятости. В результате, было принято решение заполнить пропущенные значения медианными значениями по типам дохода.

В процессе предварительной обработки данных были устранены аномалии, стандартизирована информация об образовании, удалены дубликаты и скорректированы пропуски, что увеличило точность анализа.

Пропуски в данных, особенно в столбцах 'days_employed' и 'total_income', вероятно, связаны с несовершенствами в процессе сбора данных или его техническими аспектами. Подход к заполнению этих пропусков медианными значениями является разумным, но следует помнить, что это может влиять на интерпретацию результатов.

Анализ данных показал, что предположение о свзи количества детей и возврата кредитов в срок частично подтверждено. <u>Более склонны к нарушению сроков</u> выплаты кредитов заемщики, имеющие <u>одного (<b>9,2%</b>) или двух (<b>9,5%</b>) детей</u> чем заемщики без детей (<b>7,5%</b>).

Также, можно утверждать о наличии связи между семейным положением и своевременным возвратом кредита. <u>Заемщики не состоящие в браке или проживающие в гражданском браке чаще допускают просрочки по выплатам (<b>9,8% и 9,3%</b>), чем замужние/женатые клиенты(<b>7,6%</b>)</u>.

При этом для однозначного вывода по взаимосвязи уровня дохода и нарушению сроков выплат необходимо больше данных по таким группам с доходами менее 30.000 рублей. Для нее характерна высокая доля просрочки - <b>9,1%</b>, при этом общее число заемщиков в данной категории менее <b>1,0%</b>. Клиенты с доходом от 100.001 до 200.000 также имеют высокий показатель несвоевременной выплаты долга (<b>8,6%</b>). Наиболее надежными можно считать клиентов, с уровнем дохода 200.001 - 1.000.000 руб. (<b>7,1%</b>).

<u>Доля задолженностей по кредитам на образование (<b>9,3%</b>) и автокредитам (<b>9,3%</b>) значтельно превышает аналогичный показатель по кредитам на недвижимость (<b>7,3%</b>) и проведение свадеб (<b>7,9%</b>)</u>.

Высокий процент просрочек по кредитам на автомобили и образование может отражать различные социально-экономические факторы, влияющие на эти категории заемщиков.



